# 2P-Fairdivision-IndivisibleItems

## Introduction
The aim of this project is to benchmark some alogorithm design to allocate items to two agents. This notebook focuses on same-size allocations, meaning that each agent should receive the same number of goods. As an additional constraint, items are not divisible.

This notebook is inspired by D. Marc Kilgour and Rudolf Vetschera article named *Comparing direct algoriths for two-player fair division of indivisible items - A computational study* you can found [here](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2997431)

## How to use this notebook
This notebook shows how to use algorithms and get properties of returned allocations. Several classes are defined to help you to create your own algorithms and properties :
  * Agent, which store preferences of an agent as long as some helper methods like getting the to ranked item, get the Borda score of a provided allocation ... Take a look to see what can be done !
  * Good, which is a simple wrapper to represent items. It has some nice printing  and comparison methods
  * ...
In the `properties` module, you can find a bunch of functions. They are designed to test several allocation properties such as Pareto-Optimality, Max-Min property and more.

## First steps
Let's create two agents with their preferences. As a start, we will also create 4 items to divide among agents.

In [1]:
import fairdiv


# Create some constants
NUMBER_OF_GOODS = 4

# Create items
items = [fairdiv.Good(i) for i in range(NUMBER_OF_GOODS)]

# Create agents. We will consider that item's number is their rank in preferences of the first agent
agents = [
    fairdiv.Agent(name="A", pref=items[:]),
    fairdiv.Agent(name="B", pref=[items[0], items[2], items[1], items[3]])
]

Once items and agents are created, you can use one of the available algorithm to compute an allocation. Let's use a sequential algorithm named `bottom_up`.
This algorithm generates two allocations, one when agent A is the first agent to pick an item and an other when it is agent B.

In [2]:
# Manipulate path
import os
import sys
sys.path.insert(0, os.path.join(os.getcwd(), 'fairdiv'))

# Import algorithms
import fairdiv.algorithm as algorithm


# Use bottom-up to compute two allocations
(x1, x2) = algorithm.bottom_up(agents, items)
print(x1)
print(x2)

{B: (0, 2), A: (1, 3)}
{B: (2, 3), A: (0, 1)}


Once you generated some allocations, you can test different properties with the `properties` module. Let's benchmark our allocations `x1` and `x2` to see how beautiful they are !

In [3]:
# Import module
import fairdiv.properties as properties


# Test some of them
print(properties.is_borda_envy_free(x1, agents))
print(properties.is_borda_envy_free(x2, agents))

False
False


Some properties need to get all possible allocations for a given item set size. To generate them, use `Allocation.generate_all_allocations`.

In [4]:
# Generate all allocations for our problem
A = list(fairdiv.Allocation.generate_all_allocations(agents, items))
print(A)

# Use them in properties
print(properties.is_pareto(x1, A, agents))
print(properties.is_pareto(x2, A, agents))

[{B: (1, 3), A: (0, 2)}, {B: (0, 2), A: (1, 3)}, {B: (0, 1), A: (2, 3)}, {B: (0, 3), A: (1, 2)}, {B: (2, 3), A: (0, 1)}, {B: (1, 2), A: (0, 3)}]
{B: (1, 3), A: (0, 2)}
False
True


## Test a large amount of properties
Because the number of properties to test is possibly hudge, we designed a class called `Statistics` with which you can store computed allocations and automatically test a bunch of properties easly.
First you need to create a new `Statistics` object.

In [5]:
import statistics as stats


# Create a new Statistics object for storing allocations from bottom_up
bu_stats = stats.Statistics(A, agents, {
    "is_borda_envy_free": lambda X, A, M: properties.is_borda_envy_free(X, M),
    "is_envy_free": lambda X, A, M: properties.is_envy_free(X, M),
})

Now you can add allocations to your object. Each time a new allocation is added, desired properties are tested and the result is stored. You can then print the result or save it in a file.

In [6]:
# Add allocations
bu_stats.add(x1)
bu_stats.add(x2)

# Print the result
print(bu_stats.formatted_text())

Allocations:
	Allocation: {A: (1, 3), B: (0, 2)}
		is_envy_free: True
		is_borda_envy_free: False
	Allocation: {A: (0, 1), B: (2, 3)}
		is_envy_free: True
		is_borda_envy_free: False



Lets suppose you are bad